# 大数据统计分析单元
## [拂晓工作室](https://github.com/Errrneist/Alchemist)
* 此程序为主要UI，旨在提供直观的数据分析

# 参考资料

# 导入库

In [19]:
# 导入基本库
import re
import os
import time
import random
import datetime

# 机器学习与大数据框架
import turicreate as tc
from zhon.hanzi import punctuation

# 可视化显示
import matplotlib.pyplot as plt
%matplotlib inline

# 载入数据
news_path = '../DataSets/Eastmoney/News_NLP/China/'
news_source = 'NLPCHINA20180702-1906'
news = tc.SFrame(news_path + news_source)

In [11]:
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


# [1] 热门词汇
* 通过分析TF-IDF最高值关键词的计数来发现热门方向

In [26]:
# 建立超高频略去词库
useless_words = ['金融','贷款','融资','央行','信贷','企业','财税','户数','融资贵','小微','银行']

# 建立集合字符串
keywords = []

# 集合所有关键词排名靠前的词汇
for each_news in news['keywords']:
    for word in each_news:
        if word not in useless_words:
            keywords.append(word)
            
keywords_array = tc.SArray(keywords)

# 可视化
keywords_array.show()

Materializing SArray...

Done.

Materializing SArray...

Done.